<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=eaded26d5bcccd4e668a82c9bfd0a43e5b87d27865bc8fa0551e53bd4d344207
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 11:48:14
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 2.72 K (0.02%)
Current PnL: -25.67 L (-16.58%)
CY Booked + Current PnL: -11.38 L (-7.35%)
-------------------
Total profit:  1.62 L
Total loss:  -27.30 L
-------------------
Total Booked + Current PnL: 15.60 L (12.2%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.11%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.31 L (63.88%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.06,-18.89,23.40,0.09,19128.0,-19033.0,81744.0,89.75,39.0,M-SC,2.38,253.0,-1.00,0.58,7.39,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.53,8.00,10.82,19.69,22407.0,15348.0,207090.0,-2.60,53.0,X-MC,2.47,78.0,0.68,1.47,15.35,XY25,NTT,AC
50,MASFIN,398.61,0.73,-2.56,25.25,22.05,24107.0,-2505.0,95475.0,-15.99,53.0,H-SC,6.31,164.0,-0.10,0.68,38.78,XR,ATH,FINANCE
52,NESTLEIND,1377.00,0.17,11.02,9.35,21.40,27552.0,29250.0,294676.0,3.02,57.0,X-LC,5.48,12.0,1.06,2.09,18.40,XY25,NTT,FMCG
43,ITC,452.00,0.41,-1.30,12.24,10.78,29032.0,-3128.0,237190.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.87,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.0,3.93,-20.11,82.43,45.74,159476.0,-48708.0,193469.0,6.51,61.0,H-SC,6.28,174.0,-0.31,1.37,25.83,XY24,NTT,ENGINEERING
79,VAIBHAVGBL,521.0,3.13,-25.25,115.08,60.77,157226.0,-46152.0,136623.0,57.58,53.0,H-SC,4.71,159.0,-0.29,0.97,23.04,XR,NTT,JEWELLERY
58,RECLTD,446.0,2.93,-4.06,25.85,20.74,50387.0,-8250.0,194920.0,26.46,55.0,M-MC,8.49,189.0,-0.16,1.39,6.12,XY25,NTT,FINANCE
60,REPCOHOME,880.0,2.06,-13.40,119.18,89.82,295718.0,-38378.0,248127.0,-56.78,50.0,H-SC,5.60,160.0,-0.13,1.76,31.28,XY24,NTT,FINANCE
80,VALIANTORG,838.0,1.75,-43.68,209.23,74.15,192785.0,-71465.0,92140.0,-265.94,50.0,H-SC,22.49,149.0,-0.37,0.65,18.11,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-3.85,-57.15,133.54,0.07,78491.0,-78400.0,58777.0,2054.04,69.0,L-SC,3.61,268.0,-1.00,0.42,46.18,OX40N,NTT,JEWELLERY
18,COFFEEDAY,80.0,-2.64,-41.38,130.75,35.27,87035.0,-46983.0,66566.0,-55.73,47.0,L-SC,10.06,270.0,-0.54,0.47,62.92,XR,NTT,HOTELS
6,ASIANTILES,137.0,-2.32,-4.46,90.15,81.67,80801.0,-4180.0,89630.0,8066.67,67.0,L-SC,12.14,271.0,-0.05,0.64,73.57,XR,NTT,CERAMICS
83,WHIRLPOOL,2270.0,-1.67,-24.54,141.08,81.92,122213.0,-28171.0,86627.0,-55.82,32.0,M-SC,20.66,236.0,-0.23,0.62,3.34,XR,NTT,DURABLES
41,INFY,1972.0,-1.42,13.39,18.40,34.24,65278.0,41885.0,354773.0,-11.70,68.0,X-LC,5.80,2.0,0.64,2.52,22.78,X40,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.73,-1.35,110.77,107.91,161331.0,-1999.0,145645.0,-19.74,57.0,M-SC,11.89,216.0,-0.01,1.04,8.60,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.81,-16.85,20.28,0.02,29437.0,-29405.0,145154.0,121.54,40.0,M-SC,12.02,234.0,-1.00,1.03,26.26,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.06,-18.89,23.40,0.09,19128.0,-19033.0,81744.0,89.75,39.0,M-SC,2.38,253.0,-1.00,0.58,7.39,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.19,-21.48,68.15,32.03,70543.0,-28323.0,103512.0,-15.03,63.0,H-SC,9.08,148.0,-0.40,0.74,15.94,OX40N,NTT,IT
66,SIS,528.00,-0.71,-24.04,59.64,21.26,50301.0,-26691.0,84341.0,1990.16,50.0,H-SC,4.73,166.0,-0.53,0.60,13.95,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.59,11.18,54.94,72.27,103504.0,18946.0,188394.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.05,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.59,11.18,54.94,72.27,103504.0,18946.0,188394.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.05,XR,NTT,IT
25,FINCABLES,1641.55,-0.73,-1.35,110.77,107.91,161331.0,-1999.0,145645.0,-19.74,57.0,M-SC,11.89,216.0,-0.01,1.04,8.60,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.90,-3.82,114.94,106.72,136341.0,-4717.0,118619.0,-52.34,40.0,H-SC,4.29,139.0,-0.03,0.84,18.66,AR,ATH,MISC
6,ASIANTILES,137.00,-2.32,-4.46,90.15,81.67,80801.0,-4180.0,89630.0,8066.67,67.0,L-SC,12.14,271.0,-0.05,0.64,73.57,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.80,-2.40,38.97,35.63,79708.0,-5038.0,204536.0,-16.30,46.0,H-MC,2.39,119.0,-0.06,1.45,13.87,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.05,-17.89,89.36,55.47,178497.0,-43535.0,199751.0,-27.18,32.0,X-MC,15.19,64.0,-0.24,1.42,2.35,X40N,ATH,FINANCE
22,DIXON,18931.72,-1.14,-15.44,49.08,26.07,81025.0,-30134.0,165087.0,-58.86,32.0,X-MC,11.89,56.0,-0.37,1.17,2.82,X40N,ATH,IT
8,AWL,485.00,-0.70,-24.04,102.08,53.50,233967.0,-72551.0,229200.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.63,1.80,XY24,NTT,FMCG
34,HONAUT,58357.33,-0.51,-17.90,74.57,43.32,99715.0,-29152.0,133720.0,-29.42,33.0,X-SC,5.64,90.0,-0.29,0.95,2.67,X40N,ATH,ELECTRICAL
11,BATAINDIA,2096.00,-0.26,-39.01,121.67,35.20,95488.0,-50189.0,78481.0,2.96,34.0,X-SC,16.88,92.0,-0.53,0.56,0.67,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.08,-0.21,24.74,24.48,62179.0,-528.0,251328.0,-80.87,49.0,X-SC,1.17,86.0,-0.01,1.79,22.88,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.60,-2.84,24.68,21.14,54515.0,-6452.0,220887.0,-9.65,47.0,X-MC,1.18,74.0,-0.12,1.57,24.50,XY24,NTT,PAINTS
20,DABUR,735.0,0.08,-3.40,48.76,43.70,116847.0,-8430.0,239638.0,-11.44,36.0,X-MC,2.09,72.0,-0.07,1.70,11.14,XY24,BTT,FMCG
78,UNITDSPR,1644.0,0.82,5.66,14.29,20.77,35354.0,13261.0,247405.0,-3.53,53.0,X-MC,2.14,62.0,0.38,1.76,12.97,X40N,NTT,BREWERIES
43,ITC,452.0,0.41,-1.30,12.24,10.78,29032.0,-3128.0,237190.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.87,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.26,-26.03,121.92,64.15,214591.0,-61941.0,176010.0,-56.75,37.0,X-SC,3.81,82.0,-0.29,1.25,0.49,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.26,-39.01,121.67,35.20,95488.0,-50189.0,78481.0,2.96,34.0,X-SC,16.88,92.0,-0.53,0.56,0.67,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.12,-20.05,77.00,41.51,162131.0,-52805.0,210560.0,-4.76,40.0,X-MC,7.73,61.0,-0.33,1.50,0.87,XY25,ATH,FMCG
59,RELAXO,1176.00,0.15,-47.81,191.81,52.31,145323.0,-69396.0,75764.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.41,X40N,NTT,FOOTWEAR
8,AWL,485.00,-0.70,-24.04,102.08,53.50,233967.0,-72551.0,229200.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.63,1.80,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.21,-29.36,102.21,42.84,46935.0,-19086.0,45920.0,-53.72,46.0,X-SC,34.26,83.0,-0.41,0.33,3.89,XY24,NTT,MISC
59,RELAXO,1176.00,0.15,-47.81,191.81,52.31,145323.0,-69396.0,75764.0,-44.44,42.0,X-SC,7.27,91.0,-0.48,0.54,1.41,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.26,-39.01,121.67,35.20,95488.0,-50189.0,78481.0,2.96,34.0,X-SC,16.88,92.0,-0.53,0.56,0.67,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.85,-1.63,26.14,24.08,32643.0,-2071.0,124879.0,-9.66,45.0,X-SC,6.15,89.0,-0.06,0.89,17.66,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.51,-17.90,74.57,43.32,99715.0,-29152.0,133720.0,-29.42,33.0,X-SC,5.64,90.0,-0.29,0.95,2.67,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.71,-9.22,32.97,20.71,113201.0,-34891.0,343346.0,-21.68,68.0,X-LC,3.47,1.0,-0.31,2.44,14.72,X40,ATH,IT
41,INFY,1972.00,-1.42,13.39,18.40,34.24,65278.0,41885.0,354773.0,-11.70,68.0,X-LC,5.80,2.0,0.64,2.52,22.78,X40,NTT,IT
75,TMPV,600.00,1.41,-25.35,64.74,22.98,109168.0,-57273.0,168625.0,-23.08,55.0,X-LC,6.40,3.0,-0.52,1.20,6.06,XY24,NTT,AUTO
81,VBL,671.64,-1.11,-2.99,39.85,35.67,122100.0,-9442.0,306400.0,-14.52,56.0,X-LC,3.61,4.0,-0.08,2.18,10.45,X40N,ATH,FMCG
43,ITC,452.00,0.41,-1.30,12.24,10.78,29032.0,-3128.0,237190.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.87,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.79,-37.28,111.86,32.89,54170.0,-28780.0,48427.0,-697.80,55.0,L-MC,6.81,259.0,-0.53,0.34,33.36,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-3.85,-57.15,133.54,0.07,78491.0,-78400.0,58777.0,2054.04,69.0,L-SC,3.61,268.0,-1.00,0.42,46.18,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.32,-4.46,90.15,81.67,80801.0,-4180.0,89630.0,8066.67,67.0,L-SC,12.14,271.0,-0.05,0.64,73.57,XR,NTT,CERAMICS
50,MASFIN,398.61,0.73,-2.56,25.25,22.05,24107.0,-2505.0,95475.0,-15.99,53.0,H-SC,6.31,164.0,-0.10,0.68,38.78,XR,ATH,FINANCE
13,BSOFT,831.70,1.34,-17.29,82.31,50.79,95752.0,-24322.0,116331.0,5.91,74.0,H-SC,7.64,151.0,-0.25,0.83,35.45,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.34,-17.29,82.31,50.79,95752.0,-24322.0,116331.0,5.91,74.0,H-SC,7.64,151.0,-0.25,0.83,35.45,XR,ATH,IT
56,RAJESHEXPO,518.00,-3.85,-57.15,133.54,0.07,78491.0,-78400.0,58777.0,2054.04,69.0,L-SC,3.61,268.0,-1.00,0.42,46.18,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,-0.05,-45.66,211.08,69.05,152634.0,-60755.0,72311.0,-56.18,55.0,H-SC,20.98,140.0,-0.40,0.51,11.52,SR,ATH,IT
6,ASIANTILES,137.00,-2.32,-4.46,90.15,81.67,80801.0,-4180.0,89630.0,8066.67,67.0,L-SC,12.14,271.0,-0.05,0.64,73.57,XR,NTT,CERAMICS
84,WIPRO,420.00,-0.59,11.18,54.94,72.27,103504.0,18946.0,188394.0,-4.72,73.0,M-LC,3.68,99.0,0.18,1.34,18.05,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,44.86
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.94
MC    30.76
LC    24.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.81
X40      23.01
X40N     12.96
XR        9.89
XY25      9.10
AR        8.91
OX40N     7.57
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.22
H-SC    23.80
X-LC    20.83
M-SC    11.60
X-SC     8.01
H-MC     4.81
L-SC     1.53
M-MC     1.39
M-LC     1.34
H-LC     1.11
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.21,-7.67,42.29
IT,13.17,-13.43,70.46
FINANCE,9.55,-18.61,70.20
MISC,7.26,-27.20,79.24
ELECTRICAL,5.96,-11.82,52.78
PAINTS,5.78,-13.48,30.33
INSURANCE,4.75,-1.47,36.20
PHARMA,4.07,-2.66,34.97
AUTO,2.87,-30.02,75.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3147954.0,21
AR,1316439.0,10
XR,1271098.0,13
X40,985471.0,14
X40N,861599.0,9
OX40N,728793.0,10
XY25,361537.0,6
SR,278856.0,2
MH,79181.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3627959.0,25
M-SC,1427256.0,15
X-MC,1417549.0,16
X-LC,795244.0,11
X-SC,793479.0,8
H-MC,401163.0,3
L-SC,246327.0,3
M-LC,103504.0,1
H-LC,77419.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1263005.0      6
           AR         903411.0      5
M-SC       XY24       810533.0      6
H-SC       XR         771823.0      7
X-MC       X40        488824.0      7
           XY24       405329.0      3
           X40N       338858.0      4
H-SC       OX40N      331683.0      4
M-SC       OX40N      318619.0      5
X-SC       X40N       307217.0      3
X-LC       X40        304554.0      5
X-SC       XY24       294169.0      3
H-SC       SR         278856.0      2
X-LC       X40N       215524.0      2
H-MC       AR         211270.0      2
X-SC       X40        192093.0      2
H-MC       XY24       189893.0      1
X-LC       XY24       185025.0      2
X-MC       XY25       184538.0      2
M-SC       XR         173765.0      2
L-SC       XR         167836.0      2
M-SC       AR         124339.0      2
M-LC       XR         103504.0      1
X-LC       XY25        90141.0      2
H-SC       MH          79181.0      1
L-SC       OX40N       78491.0      1
H-LC       AR          77419.0      1
L-MC       XR          54170.0      1
M-MC       XY25        50387.0      1
L-LC       XY25        36471.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
